In [1]:
import os

In [2]:
%pwd

'd:\\Deep-Learning-Project\\Deepl-Learning-CNN-Project\\research'

In [3]:
os.chdir('../')


In [4]:
%pwd

'd:\\Deep-Learning-Project\\Deepl-Learning-CNN-Project'

In [25]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

#### update components



In [26]:
from src.CNN_Classification.constants import *
from src.CNN_Classification.utils.common import read_yaml, create_directories
import tensorflow as tf

In [27]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "MRI DATASET") 
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [28]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [31]:
import tensorflow as tf
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator


class Training:
    def __init__(self, config):
        self.config = config
        self.model = None
        self.train_generator = None
        self.test_generator = None

    def load_model(self):
        """Loads the updated base model for training."""
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

    def train_test_generator(self):
        """Creates data generators for training and testing data."""
        datagenerator_kwargs = dict(rescale=1. / 255)  # Normalize pixel values

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        # Training data generator (with or without augmentation)
        if self.config.params_is_augmentation:
            train_datagenerator = ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = ImageDataGenerator(**datagenerator_kwargs)

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            shuffle=True,
            **dataflow_kwargs
        )

        # Test data generator
        test_datagenerator = ImageDataGenerator(**datagenerator_kwargs)
        self.test_generator = test_datagenerator.flow_from_directory(
            directory=self.config.testing_data,
            shuffle=False,
            **dataflow_kwargs
        )

    def train(self):
        """Trains the model and saves it after training."""
        steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=steps_per_epoch
        )

        self.save_model(self.config.trained_model_path, self.model)

    def evaluate(self):
        """Evaluates the trained model on test data."""
        test_steps = self.test_generator.samples // self.test_generator.batch_size
        loss, accuracy = self.model.evaluate(self.test_generator, steps=test_steps)
        print(f"Test Accuracy: {accuracy:.4f}")
        print(f"Test Loss: {loss:.4f}")

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        """Saves the trained model."""
        model.save(path)



##### update pipeline

In [33]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    print(f"An error occurred: {e}")

[2025-03-16 15:54:44,494: INFO: common: yaml file:config\config.yaml loaded successfully]
[2025-03-16 15:54:44,497: INFO: common: yaml file:params.yaml loaded successfully]
[2025-03-16 15:54:44,499: INFO: common: created directory at: artifacts]
[2025-03-16 15:54:44,501: INFO: common: created directory at: artifacts\training]
An error occurred: 'Training' object has no attribute 'get_base_model'
